# Init

In [1]:
import hnswlib
import numpy as np
import sqlite3
import pickle
#import tqdm
from tqdm.auto import tqdm, trange
import time

/home/danielperry/repos/steam-natural-search/03_hnsw-index/.venv_03/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Open the database
conn = sqlite3.connect('../steam_instructor-xl.db')

In [9]:
# Helper functions

def get_any_description_embedding():
    c = conn.cursor()

    c.execute(f'''
        SELECT embedding FROM description_embeddings LIMIT 1
    ''')
    results = c.fetchone()[0]

    c.close()

    return pickle.loads(results)

def get_description_embeddings_for_appid(appid):
    c = conn.cursor()

    c.execute(f'''
        SELECT embedding FROM description_embeddings WHERE appid = ?
    ''', (appid,))
    results = c.fetchone()[0]

    c.close()

    return pickle.loads(results)

def get_review_embeddings_for_appid(appid):
    c = conn.cursor()

    c.execute(f'''
        SELECT recommendationid, embedding
        FROM review_embeddings
        WHERE appid = ?
    ''', (appid,))
    results = c.fetchall()

    c.close()

    return {recommendationid: pickle.loads(embedding) for recommendationid, embedding in results}

def get_count_appids_with_description_embeddings():
    c = conn.cursor()

    c.execute(f'''
        SELECT COUNT(DISTINCT appid) FROM description_embeddings
    ''')
    results = c.fetchone()[0]

    c.close()

    return results

def get_count_appids_with_review_embeddings():
    c = conn.cursor()

    c.execute(f'''
        SELECT COUNT(DISTINCT appid) FROM review_embeddings
    ''')
    results = c.fetchone()[0]

    c.close()

    return results

def get_all_description_embeddings_generator(page_size):
    c = conn.cursor()

    c.execute(f'''
        SELECT appid, embedding FROM description_embeddings
    ''')

    while True:
        results = c.fetchmany(page_size)
        if not results:
            break
        for appid, embedding in results:
            yield appid, pickle.loads(embedding)

    c.close()

def get_batched_description_embeddings_generator(page_size):
    c = conn.cursor()

    c.execute(f'''
        SELECT appid, embedding FROM description_embeddings
    ''')

    while True:
        results = c.fetchmany(page_size)
        if not results:
            break
        yield [(appid, pickle.loads(embedding)) for appid, embedding in results]

    c.close()

def get_all_review_embeddings_generator(page_size):
    c = conn.cursor()

    c.execute(f'''
        SELECT appid, embedding FROM review_embeddings
    ''')

    while True:
        results = c.fetchmany(page_size)
        if not results:
            break
        for appid, embedding in results:
            yield appid, pickle.loads(embedding)

    c.close()

def get_appids_with_reviews():
    c = conn.cursor()

    c.execute(f'''
        SELECT DISTINCT appid FROM review_embeddings
    ''')

    results = c.fetchall()

    c.close()

    return [appid for appid, in results]

def mean_pooling(embeddings):
    return np.sum(embeddings, axis=0) / len(embeddings)

def get_pooled_description_embedding_for_appid(appid):
    return mean_pooling(get_description_embeddings_for_appid(appid))

def get_pooled_review_embeddings_for_appid(appid):
    all_review_embeddings = get_review_embeddings_for_appid(appid)
    flat_review_embeddings = [review_embedding for review_id in all_review_embeddings for review_embedding in all_review_embeddings[review_id]]
    return mean_pooling(flat_review_embeddings)

def get_reviews_by_appid_batched(page_size=1000):
    appids = get_appids_with_reviews()

    for i in range(0, len(appids), page_size):
        yield [(appid, get_pooled_review_embeddings_for_appid(appid)) for appid in appids[i:i+page_size]]



def get_index_dimension():
    return len(get_any_description_embedding()[0])

# Recall Testing

## Description Embeddings

### `cosine` Distance

**Results Table**  

| k  | ef  | M  | recall | time to build | query time (batched) | query time (individual) |
|----|-----|----|--------|---------------|----------------------|-------------------------|
| 10 | 200 | 54 | 55-56% | N/A | N/A | N/A |
| 10 | 400 | 48 | 77%  | N/A | N/A | N/A |
| 10 | 800 | 48 | 88% | `4m15s`* (?) | N/A | N/A |
| 10 | 2000 | 32 | 93% | `2m22s` | `4s` total / `2ms` per query | `4.8ms` per query |
| 10 | 2000 | 48 | 95% | `7m53s`* | `11s`* total / `5.5ms` per query | N/A |
| 10 | 5000 | 28 | 97.7% | `4m14s` | `8.8s` total / `4.4ms` per query | `10.5ms` per query |

<!-- not enough queries made -->
<!--| 10 | 200 | 48 | ~56%    |-->

(?) - Seems suspect  
\* - Numbers from M1 MBP, slower than rest (which is Ryzen 7 5800X)

### `l2` Distance

**Results Table**

| k  | ef  | M  | recall | time to build | query time (batched) | query time (individual) |
|----|-----|----|--------|---------------|----------------------|-------------------------|
| 10 | 2000 | 28 | 93% | `2m15s` | `3.2s` total | `4.44ms` per query |

## Review Embeddings

### `cosine` Distance

**Results Table**  

| k  | ef  | M  | recall | time to build | query time (batched) | query time (individual) |
|----|-----|----|--------|---------------|----------------------|-------------------------|
| 10 | 2000 | 28 | 57% | `1m6s` | `1.14s` | `2.14ms` |
| 10 | 2000 | 48 | 57% | `1m10s` | `1.18s` |  |
| 10 | 10000 | 28 | 88% | `3m22s` | `7.9s` | `11.66ms` |
| 10 | 10000 | 48 | 89% | `3m26s` | `8s` | `11.9ms` |
| 10 | 15000 | 28 | 92% | `4m13s` | `10s` | `15.4ms` |

## Code

In [4]:
num_queries = 2000

In [5]:
dim = get_index_dimension()

indexing = 'review'

if indexing == 'description':
  num_elements = get_count_appids_with_description_embeddings()
elif indexing == 'review':
  num_elements = get_count_appids_with_review_embeddings()
else:
  raise Exception('Invalid indexing')

k = 10 # Number of nearest neighbors

alg = 'cosine' # can be 'l2', 'ip' or 'cosine'

# ef - the size of the dynamic list for the nearest neighbors (used during the search).
# Higher = more accurate, but slower
ef = 15000

# M -  the number of bi-directional links created for every new element during construction.
# 12-48 is good for most cases. 
# Highly related to dimensionality of the data.
#   dim = 4 -> M = 6 - 12
#   dim = 512 -> M = 48 - 64
M = 28

In [6]:
hnsw_index = hnswlib.Index(space=alg, dim=dim)
hnsw_index.init_index(max_elements=num_elements, ef_construction=ef, M=M)
hnsw_index.set_ef(ef)

In [7]:
bf_index = hnswlib.BFIndex(space=alg, dim=dim)
bf_index.init_index(max_elements=num_elements)

bf_index_initialized = False

### Build Description Embeddings Index

In [8]:
bar = tqdm(total=num_elements, smoothing=0.1)

if indexing == 'description':
    for batch in get_batched_description_embeddings_generator(1000):
        appids, embeddings = zip(*batch)
        embeddings = [mean_pooling(embedding) for embedding in embeddings]

        hnsw_index.add_items(embeddings, appids)

        if not bf_index_initialized:
            bf_index.add_items(embeddings, appids)
        
        bar.update(len(batch))
elif indexing == 'review':
    for batch in get_reviews_by_appid_batched(page_size=1000):
        appids, embeddings = zip(*batch)

        hnsw_index.add_items(embeddings, appids)

        if not bf_index_initialized:
            bf_index.add_items(embeddings, appids)
        
        bar.update(len(batch))

bar.close()
bf_index_initialized = True

100%|██████████| 74155/74155 [04:13<00:00, 291.95it/s] 


### Test

In [12]:

# Time batched queries (num_queries queries at a time)
query_data = np.float32(np.random.random((num_queries, dim)))

print("Querying hnsw index...", end="")
hnsw_begin = time.perf_counter()
labels_hnsw, distances_hnsw = hnsw_index.knn_query(query_data, k)
hnsw_end = time.perf_counter()
print("took", hnsw_end - hnsw_begin, "seconds")

print("Querying brute force index...")
labels_bf, distances_bf = bf_index.knn_query(query_data, k)

correct = sum([1 for i in range(num_queries) for label in labels_hnsw[i] if label in labels_bf[i]])

print("recall is :", float(correct)/(k*num_queries))

Querying hnsw index...took 9.944127280963585 seconds
Querying brute force index...
recall is : 0.91985


In [11]:
# Time individual queries
# One timer on outer loop: 0.056638814979000016 seconds
# Timer on inner loop:     0.05581076550566649 seconds
query_data = np.float32(np.random.random((num_queries, dim)))

total_time = 0.0

for i in trange(num_queries):
    query = query_data[i]

    hnsw_begin = time.perf_counter()
    _ = hnsw_index.knn_query(query, k)
    hnsw_end = time.perf_counter()

    total_time += hnsw_end - hnsw_begin

print("Average time per query:", total_time / num_queries, "seconds")

100%|██████████| 2000/2000 [00:30<00:00, 64.68it/s]

Average time per query: 0.01538519425864797 seconds
